<a href="https://colab.research.google.com/github/AnnaVashuk/mts/blob/main/%D0%94%D0%97_%D1%88%D0%B0%D0%B4_%D0%BC%D0%BB_%D0%BA%D0%B5%D0%B9%D1%81%D1%8B_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
import sys
import time
import torch
from pyod.models.pca import PCA as PCA_PyOD
from pyod.utils.data import evaluate_print
from pyod.utils.data import generate_data
from pytod.models.pca import PCA
from pytod.utils.utility import validate_device
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from pyod.models.lof import LOF as LOF_PyOD
from pytod.models.lof import LOF
from pyod.models.knn import KNN as KNN_PyOD
from pytod.models.knn import KNN
from pyod.models.abod import ABOD as ABOD_PyOD
from pytod.models.abod import ABOD
from pyod.models.hbos import HBOS as HBOS_PyOD
from pytod.models.hbos import HBOS
from pytod.utils.utility import validate_device
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Задание: сделать сравнение быстродействия PYOD vs PYTOD для минимум 5 алгоритмов

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/creditcard.csv")

In [ ]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


# Preprocessing



In [ ]:
df.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,1.168375e-15,3.416908e-16,-1.379537e-15,2.074095e-15,9.604066e-16,1.487313e-15,-5.556467e-16,1.213481e-16,-2.406331e-15,...,1.654067e-16,-3.568593e-16,2.578648e-16,4.473266e-15,5.340915e-16,1.683437e-15,-3.660091e-16,-1.227390e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [ ]:
X = df.drop(['Class'], axis=1)
y = df['Class']

y.value_counts()

Class
0    284315
1       492
Name: count, dtype: int64

# Train and test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train, X_test, y_train, y_test = X_train.head(8000), X_test.head(2000), y_train.head(8000), y_test.head(2000)

sm = SMOTE(random_state=42)
X_train, y_train = sm.fit_resample(X_train, y_train)

In [ ]:
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
X_test = X_test.to_numpy()
y_test = y_test.to_numpy()

In [ ]:
training_times = {}
prediction_times = {}
logs = {}

# PCA

In [ ]:
# PyOD PCA
clf = PCA_PyOD(n_components=5)
start = time.time()
clf.fit(X_train)
end = time.time()
pyod_time_PCA = end - start
training_times['PyOD_PCA'] = pyod_time_PCA
print('PyOD PCA execution time', pyod_time_PCA)
logs['PyOD_PCA'] = clf

X_train, y_train, X_test, y_test = torch.from_numpy(X_train), \
                                   torch.from_numpy(y_train), \
                                   torch.from_numpy(X_test), \
                                   torch.from_numpy(y_test)
# TOD PCA
device = validate_device(0)
clf = PCA(n_components=5, device=device)
start = time.time()
clf.fit(X_train)
end = time.time()
tod_time_PCA = end - start
training_times['TOD_PCA'] = tod_time_PCA
print('TOD PCA execution time', tod_time_PCA)
logs['TOD_PCA'] = clf

PyOD PCA execution time 0.08056330680847168
TOD PCA execution time 1.1920225620269775


# LOF

In [ ]:
# PyOD LOF
clf = LOF_PyOD(n_neighbors=5)
start = time.time()
clf.fit(X_train)
end = time.time()
pyod_time_LOF = end - start
training_times['PyOD_LOF'] = pyod_time_LOF
print('PyOD LOF execution time', pyod_time_LOF)
logs['PyOD_LOF'] = clf

# TOD LOF
device = 'cpu'
clf = LOF(n_neighbors=5, batch_size=10000, device=device)
start = time.time()
clf.fit(X_train)
end = time.time()
tod_time_LOF = end - start
training_times['TOD_LOF'] = tod_time_LOF
print('TOD LOF execution time', tod_time_LOF)
logs['TOD_LOF'] = clf


PyOD LOF execution time 1.3431029319763184
n_batches 1
n_batches 1
[(0, 10000), (10000, 15976)]
[(0, 10000), (10000, 15976)]
TOD LOF execution time 1.8134675025939941


# KNN

In [ ]:
# PyOD KNN
clf = KNN_PyOD(n_neighbors=5)
start = time.time()
clf.fit(X_train)
end = time.time()
pyod_time_KNN = end - start
training_times['PyOD_KNN'] = pyod_time_KNN
print('PyOD KNN execution time', pyod_time_KNN)
logs['PyOD_KNN'] = clf

# TOD KNN
device = 'cpu'
clf = KNN(n_neighbors=5, batch_size=None, device=device)
start = time.time()
clf.fit(X_train)
end = time.time()
tod_time_KNN = end - start
training_times['TOD_KNN'] = tod_time_KNN
print('TOD KNN execution time', tod_time_KNN)
logs['TOD_KNN'] = clf

PyOD KNN execution time 1.5081768035888672
TOD KNN execution time 3.9349253177642822


# HBOS

In [ ]:
# PyOD HBOS
clf = HBOS_PyOD()
start = time.time()
clf.fit(X_train)
end = time.time()
pyod_time_HBOS = end - start
training_times['PyOD_HBOS'] = pyod_time_HBOS
print('PyOD HBOS execution time', pyod_time_HBOS)
logs['PyOD_HBOS'] = clf

# TOD HBOS
device = 'cpu'
clf = HBOS(device=device)
start = time.time()
clf.fit(X_train)
end = time.time()
tod_time_HBOS = end - start
training_times['TOD_HBOS'] = tod_time_HBOS
print('TOD HBOS execution time', tod_time_HBOS)
logs['TOD_HBOS'] = clf

PyOD HBOS execution time 2.17568039894104
TOD HBOS execution time 0.04825568199157715


/usr/local/lib/python3.10/dist-packages/pytod/models/hbos.py:107: UserWarning: torch.searchsorted(): boundary tensor is non-contiguous, this will lower the performance due to extra data copy when converting non-contiguous tensor to contiguous, please use contiguous boundary tensor if possible. This message will only appear once per program. (Triggered internally at ../aten/src/ATen/native/BucketizationUtils.h:38.)
  bin_inds = torch.bucketize(X[:, i], bin_edges[:, i])


# ABOD

In [ ]:

# PyOD ABOD
clf = ABOD_PyOD(n_neighbors=5)
start = time.time()
clf.fit(X_train)
end = time.time()
pyod_time_ABOD = end - start
training_times['PyOD_ABOD'] = pyod_time_ABOD
print('PyOD ABOD execution time', pyod_time_ABOD)
logs['PyOD_ABOD'] = clf

# TOD ABOD
device = validate_device(0)
clf = ABOD(n_neighbors=5, batch_size=10000, device=device)
start = time.time()
clf.fit(X_train)
end = time.time()
tod_time_ABOD = end - start
training_times['TOD_ABOD'] = tod_time_ABOD
print('TOD ABOD execution time', tod_time_ABOD)
logs['TOD_ABOD'] = clf

PyOD ABOD execution time 4.229866981506348
n_batches 1
n_batches 1
[(0, 10000), (10000, 15976)]
[(0, 10000), (10000, 15976)]
TOD ABOD execution time 2.8122308254241943


In [ ]:
results = pd.DataFrame({
    'Algorithm': list(training_times.keys()),
    'Training Time': list(training_times.values())
})

In [ ]:
def get_prediction_time(model, X_test):
    start = time.time()
    predictions = model.decision_function(X_test)
    end = time.time()
    return end - start, predictions

for name, model in logs.items():
    pred_time, _ = get_prediction_time(model, X_test)
    prediction_times[name] = pred_time

results['Prediction Time'] = results['Algorithm'].map(prediction_times)

print(results)

   Algorithm  Training Time  Prediction Time
0   PyOD_PCA       0.080563         0.008019
1    TOD_PCA       1.192023         0.000004
2   PyOD_LOF       1.343103         0.547782
3    TOD_LOF       1.813468         0.000005
4   PyOD_KNN       1.508177         0.578567
5    TOD_KNN       3.934925         0.000004
6  PyOD_HBOS       2.175680         0.002097
7   TOD_HBOS       0.048256         0.000004
8  PyOD_ABOD       4.229867         0.913608
9   TOD_ABOD       2.812231         0.000005
